In [ ]:
# Install & Mount (run once)
!pip install --quiet albumentations ipywidgets pillow numpy matplotlib

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ——————— USER CONFIGURATION ———————
transform_type = 'fog'    # 'fog' or 'snow'
input_folder   = '/content/images'
# —————————————————————————————————————

import os, random
import numpy as np
from PIL import Image
import albumentations as A
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from collections import Counter

# ——— Build the full grid of candidate hyperparams ———
fog_candidates = [
    {"fog_coef_lower": l, "fog_coef_upper": u, "alpha_coef": a}
    for l in np.arange(0.0, 1.0, 0.2)
    for u in np.arange(0.2, 1.2, 0.2)
    for a in np.arange(0.02, 0.22, 0.02)
    if l < u
]
snow_candidates = [
    {"brightness_coeff": b, "snow_point_lower": lo, "snow_point_upper": hi}
    for b in np.arange(1.0, 3.1, 0.25)
    for lo in np.arange(0.0, 0.6, 0.1)
    for hi in np.arange(0.2, 0.9, 0.1)
    if lo < hi
]

if transform_type.lower() == 'fog':
    all_params     = fog_candidates
    make_transform = lambda **p: A.RandomFog(**p, p=1.0)
else:
    all_params     = snow_candidates
    make_transform = lambda **p: A.RandomSnow(**p, p=1.0)

# ——— Gather your images ———
file_list = sorted([
    f for f in os.listdir(input_folder)
    if os.path.isfile(os.path.join(input_folder, f))
    and f.lower().rsplit('.',1)[-1] in ('jpg','jpeg','png','bmp')
])
# Pre-sample 9 random candidates per image
picks_list = [ random.sample(all_params, 9) for _ in file_list ]

# ——— Interactive selection state ———
current_idx = 0
selected    = []

# ——— Helper to show one grid ———
def show_grid(idx):
    clear_output(wait=True)
    fname = file_list[idx]
    img   = Image.open(os.path.join(input_folder, fname)).convert('RGB')
    picks = picks_list[idx]

    # Plot 3×3
    fig, axes = plt.subplots(3, 3, figsize=(9,9))
    for ax, (i, params) in zip(axes.ravel(), enumerate(picks, start=1)):
        tr  = make_transform(**params)
        arr = np.array(img)[:,:,::-1]              # RGB→BGR
        out = tr(image=arr)['image'][:,:,::-1]     # BGR→RGB
        ax.imshow(out)
        ax.set_title(
            f"{i}: " + ", ".join(f"{k}={v:.2f}" for k,v in params.items()),
            fontsize=6
        )
        ax.axis('off')
    fig.suptitle(f"{transform_type.title()} options for {fname}", fontsize=14)
    plt.tight_layout(rect=[0,0,1,0.95])
    plt.show()

    # Dropdown & Button
    dropdown = widgets.Dropdown(options=list(range(1,10)), description='Pick:')
    button   = widgets.Button(description='Submit & Next', button_style='primary')
    display(widgets.HBox([dropdown, button]))

    # Button callback
    def on_click(b):
        nonlocal dropdown, button, idx
        sel = dropdown.value
        selected.append(tuple(sorted(picks_list[idx][sel-1].items())))
        button.disabled = True
        proceed()

    button.on_click(on_click)

# ——— Proceed to next or finish ———
def proceed():
    global current_idx
    current_idx += 1
    if current_idx < len(file_list):
        show_grid(current_idx)
    else:
            # … after the last image is done …
        clear_output(wait=True)

        # replace the old Counter‐mode block with:
        import pandas as pd
        df = pd.DataFrame(selected)

        median_params = df.median().to_dict()
        print("🔎 Median hyperparameters across your picks:")
        for k, v in median_params.items():
            print(f"  {k}: {v:.3f}")

        # optional: also show mean
        mean_params = df.mean().to_dict()
        print("\n🔎 Mean hyperparameters across your picks:")
        for k, v in mean_params.items():
            print(f"  {k}: {v:.3f}")


# ——— Kick off the loop ———
print(f"Found {len(file_list)} images. Starting with image #1.")
show_grid(current_idx)


In [ ]:
import pandas as pd

# Suppose you have:
# selected = [
#   (('fog_coef_lower', 0.2), ('fog_coef_upper', 0.4), ('alpha_coef', 0.04)),
#   (('fog_coef_lower', 0.4), ('fog_coef_upper', 0.6), ('alpha_coef', 0.06)),
#    ...
# ]

# Convert back to dicts:
selected_dicts = [dict(pairs) for pairs in selected]

# Now build your DataFrame
df = pd.DataFrame(selected_dicts)

# And compute medians (or means)
median_params = df.median().to_dict()
print("Median hyperparameters:")
for k, v in median_params.items():
    print(f"  {k}: {v:.3f}")

mean_params = df.mean().to_dict()
print("\nMean hyperparameters:")
for k, v in mean_params.items():
    print(f"  {k}: {v:.3f}")


In [ ]:
# ——— SWITCH TO SNOW MODE ———
transform_type = 'snow'

# rebuild params & transform fn
all_params     = snow_candidates
make_transform = lambda **p: A.RandomSnow(**p, p=1.0)

# resample 9 candidates per image
picks_list = [ random.sample(all_params, 9) for _ in file_list ]

# reset your state
current_idx = 0
selected    = []

print(f"Running Snow selection for {len(file_list)} images…")
show_grid(current_idx)


In [ ]:
import pandas as pd

# Suppose you have:
# selected = [
#   (('fog_coef_lower', 0.2), ('fog_coef_upper', 0.4), ('alpha_coef', 0.04)),
#   (('fog_coef_lower', 0.4), ('fog_coef_upper', 0.6), ('alpha_coef', 0.06)),
#    ...
# ]

# Convert back to dicts:
selected_dicts = [dict(pairs) for pairs in selected]

# Now build your DataFrame
df = pd.DataFrame(selected_dicts)

# And compute medians (or means)
median_params = df.median().to_dict()
print("Median hyperparameters:")
for k, v in median_params.items():
    print(f"  {k}: {v:.3f}")

mean_params = df.mean().to_dict()
print("\nMean hyperparameters:")
for k, v in mean_params.items():
    print(f"  {k}: {v:.3f}")
